In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")

import torch
import torch.nn as nn
import torch.nn.functional as F
import math

DATA_PATH = "../../data"

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

from src.log_mock import PrintLog
log = PrintLog()

import wandb

wandb.init(mode="disabled")

In [ ]:
wapi = wandb.Api()
runs = wapi.runs("bayes/rxrx1")

In [ ]:
for run in runs:
    print(run.name, run.summary.keys())

In [ ]:
import plotly.express as px
import pandas as pd
import dateutil
import datetime

def create_plot_data_for_run(run):
    parts = run.name.split("-")
    if len(parts) > 2:
        model_name = parts[0] + "-" + parts[1]
    else:
        model_name = parts[0]

    return {
        "model": model_name,
        "ood accuracy": run.summary["test_results"]["accuracy"],
        "ood log likelihood": run.summary["test_results"]["log_likelihood"],
        "ood ece": run.summary["test_results"]["ece"],
        "ood sece": run.summary["test_results"]["sece"],
        "id accuracy": run.summary["id_test_results"]["accuracy"],
        "id log likelihood": run.summary["id_test_results"]["log_likelihood"],
        "id ece": run.summary["id_test_results"]["ece"],
        "id sece": run.summary["id_test_results"]["sece"],
    }

def plot(data, value):
    plot = px.box(data, x="model", y=value, color="model")
    return plot

def pareto_plot(data, x, y):
    plot = px.scatter(data, x=x, error_x=f"{x}_std", y=y, error_y=f"{y}_std", color="model")
    return plot

def build_data(runs):
    rows = []
    for run in runs:
        if dateutil.parser.parse(run.created_at) < datetime.datetime(2023, 3, 10, 10, 0):
            continue
        if run.state != "finished":
            continue
        if "old" in run.tags:
            print("Skipping old run " + run.name)
            continue
        if "test_results" not in run.summary:
            print("Skipping crashed run " + run.name)
            continue
        rows.append(create_plot_data_for_run(run))
    return pd.DataFrame.from_dict(rows)

def aggregate_data(data):
    aggregated_data = data.groupby(["model"]).agg({
        "model": "first",
        "ood accuracy": ["mean", "sem"],
        "ood log likelihood": ["mean", "sem"], 
        "ood sece": ["mean", "sem"],
        "ood ece": ["mean", "sem"],
        "id accuracy": ["mean", "sem"],
        "id log likelihood": ["mean", "sem"], 
        "id sece": ["mean", "sem"],
        "id ece": ["mean", "sem"],
    })
    aggregated_data.columns = [a[0] + "_std" if a[1] == "sem" else a[0] for a in aggregated_data.columns.to_flat_index()]
    aggregated_data["ood accuracy_std"] *= 2.0
    aggregated_data["ood log likelihood_std"] *= 2.0
    aggregated_data["ood sece_std"] *= 2.0
    aggregated_data["ood ece_std"] *= 2.0
    aggregated_data["id accuracy_std"] *= 2.0
    aggregated_data["id log likelihood_std"] *= 2.0
    aggregated_data["id sece_std"] *= 2.0
    aggregated_data["id ece_std"] *= 2.0
    return aggregated_data

In [ ]:
data = aggregate_data(build_data(runs))

In [ ]:
data

In [ ]:
pareto_plot(data, "ood accuracy", "ood sece")

In [ ]:
pareto_plot(data, "id accuracy", "id sece")

In [ ]:
data.to_csv(sep=",", header=True)

In [ ]:
algo_names = [
    ("map_1", "MAP"),
    ("map_5", "Deep Ensemble"),
    ("mcd_1", "MCD"),
    ("mcd_5", "MultiMCD"),
    ("swag_1", "SWAG"),
    ("swag_5", "MultiSWAG"),
    # ("swag_ll-1", "LL SWAG"),
    ("laplace-1", "LL Laplace"),
    # ("laplace-5", "LL MultiLaplace"),
    ("bbb_1", "LL BBB ($\\sigma=1.0, \\lambda=1.0$)"),
    ("bbb_prior0.5_1", "LL BBB ($\\sigma=0.5, \\lambda=1.0$)"),
    ("bbb_prior0.1_1", "LL BBB ($\\sigma=0.1, \\lambda=1.0$)"),
    ("bbb_kl0.2_1", "LL BBB ($\\sigma=1.0, \\lambda=0.2$)"),
    # ("bbb_5", "LL MultiBBB"),
    ("rank1_1", "Rank-1 VI"),
    ("ll_ivon_1", "LL iVON"),
    # ("ll_ivon_5", "LL MultiiVON"),
    ("svgd_1", "SVGD"),
    # ("ll_svgd-1", "LL SVGD"),
    ("sngp", "SNGP"),
]

def num(value, std, best=None, ty=None):
    value = float(value)
    std = float(std)
    num_string = f"{value:.3f} \\pm {std:.3f}"

    if best is None or ty is None:
        return f"${num_string}$"

    if ty == "max":
        if value >= best:
            num_string = f"\\bm{{{num_string}}}"
    elif ty == "min":
        if value <= best:
            num_string = f"\\bm{{{num_string}}}"
    elif ty == "zero":
        if abs(value) <= best:
            num_string = f"\\bm{{{num_string}}}"
    return f"${num_string}$"

def col_name(name, align):
    return f"\\multicolumn{{1}}{{{align}}}{{{name}}}"

def create_table(data, prefix):
    print("\\begin{tabular}{l|rrrrrr}")
    print(f"    {col_name('Model', 'l')} & {col_name('i.d. Accuracy', 'c')} & {col_name('i.d. ECE', 'c')} & {col_name('i.d. sECE', 'c')}& {col_name('o.o.d. Accuracy', 'c')} & {col_name('o.o.d. ECE', 'c')} & {col_name('o.o.d. sECE', 'c')} \\\\")
    print("    \\hline")

    best_acc, best_acc_std = 0, 0
    best_ece, best_ece_std = 1000, 0
    best_sece, best_sece_std = 1000, 0

    best_id_acc, best_id_acc_std = 0, 0
    best_id_ece, best_id_ece_std = 1000, 0
    best_id_sece, best_id_sece_std = 1000, 0

    for algo, name in algo_names:
        row = data[data["model"] == algo]
        if (row.empty):
            continue
        
        if float(row[prefix + "ood accuracy"]) > best_acc:
            best_acc = float(row[prefix + "ood accuracy"])
            best_acc_std = float(row[prefix + "ood accuracy_std"])
        
        if float(row[prefix + "ood ece"]) < best_ece:
            best_ece = float(row[prefix + "ood ece"])
            best_ece_std = float(row[prefix + "ood ece_std"])
        
        if abs(float(row[prefix + "ood ece"])) < best_sece:
            best_sece = abs(float(row[prefix + "ood sece"]))
            best_sece_std = float(row[prefix + "ood sece_std"])
        
        if float(row[prefix + "id accuracy"]) > best_acc:
            best_id_acc = float(row[prefix + "id accuracy"])
            best_id_acc_std = float(row[prefix + "id accuracy_std"])
        
        if float(row[prefix + "id ece"]) < best_ece:
            best_id_ece = float(row[prefix + "id ece"])
            best_id_ece_std = float(row[prefix + "id ece_std"])
        
        if abs(float(row[prefix + "id sece"])) < best_sece:
            best_id_sece = abs(float(row[prefix + "id sece"]))
            best_id_sece_std = float(row[prefix + "id sece_std"])

    best_acc -= best_acc_std
    best_ece += best_ece_std
    best_sece = abs(best_sece) + best_sece_std

    best_id_acc -= best_id_acc_std
    best_id_ece += best_id_ece_std
    best_id_sece = abs(best_id_sece) + best_id_sece_std

    for algo, name in algo_names:
        row = data[data["model"] == algo]
        if row.empty:
            continue
        print(f"    {name} & {num(row[prefix + 'id accuracy'], row[prefix + 'id accuracy_std'], best_id_acc, 'max')} & {num(row[prefix + 'id ece'], row[prefix + 'id ece_std'], best_id_ece, 'min')} & {num(row[prefix + 'id sece'], row[prefix + 'id sece_std'], best_id_sece, 'zero')} & {num(row[prefix + 'ood accuracy'], row[prefix + 'ood accuracy_std'], best_acc, 'max')} & {num(row[prefix + 'ood ece'], row[prefix + 'ood ece_std'], best_ece, 'min')} & {num(row[prefix + 'ood sece'], row[prefix + 'ood sece_std'], best_sece, 'zero')} \\\\")
    print("\\end{tabular}")
create_table(data, "")